# Bluecap news competition

## 1. Setup and basic checks

### 1.1. Library version check

In [1]:
# Check the versions of libraries

# Python version
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy as sp
print('scipy: {}'.format(sp.__version__))
# numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib
import matplotlib as mpl
print('matplotlib: {}'.format(mpl.__version__))
# pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
import sklearn as skl
print('sklearn: {}'.format(skl.__version__))

Python: 3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
scipy: 1.0.0
numpy: 1.14.0
matplotlib: 2.1.2
pandas: 0.22.0
sklearn: 0.19.1


### 1.2. Load libraries

In [2]:
# Load libraries

from pandas.plotting import scatter_matrix
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
#from itertools import chain
import ast
import nltk
from nltk.corpus import stopwords # Language processing library - stopwords
from nltk.tokenize import word_tokenize # Word tokenizer for titles and corpora

## 2. Loading the datasets

We will be using pandas to read the csv's and do some preliminary data exploration



In [3]:
trainUrl = "train.csv"
trainSet = pd.read_csv(trainUrl)

testUrl = "test.csv"
testSet = pd.read_csv(testUrl)

## 3. Exploration

### 3.1. Shape and raw data

In [4]:
# First we will look at the shape of the datasets:

print(trainSet.shape)
print(testSet.shape)

(14637, 7)
(5225, 6)


In [5]:
# Now let's print some of the first and last rows to see how the data looks like:
## Train set
trainSet.head(5)

,id,keywords,summary,title,text,date,flag
0,1,"[optimismo, su, banco, tipos, eleva, que, el, ...",Los nueve consejeros del Banco de Inglaterra h...,El Banco de Inglaterra mantiene tipos y eleva ...,Los nueve consejeros del Banco de Inglaterra h...,2016-11-03,0
1,2,"[por, en, hasta, y, su, del, primer, el, que, ...","El primer banco de Italia, que completó con éx...",UniCredit duplica su beneficio en el primer tr...,"El primer banco de Italia, que completó con éx...",2017-05-11,1
2,3,"[se, su, que, checa, presenta, todocamino, el,...","Ocupará el lugar del actual Yeti y con él, Sko...",Skoda Karoq: así se presenta el todocamino com...,"Ocupará el lugar del actual Yeti y con él, Sko...",2017-04-28,0
3,4,"[y, los, en, la, que, millones, hasta, el, ban...",Bankia obtuvo un beneficio neto atribuido de 4...,"Bankia gana el 13,4% menos en el semestre, en ...",Bankia obtuvo un beneficio neto atribuido de 4...,2016-07-22,0
4,5,"[la, coste, su, tres, se, las, consigue, ha, m...",La rentabilidad media en sus bonos a tres años...,El Tesoro consigue colocar a coste cero su deu...,La rentabilidad media en sus bonos a tres años...,2016-05-19,0


In [6]:
## Test set
testSet.head(5)

,id,keywords,summary,title,text,date
0,1,"[despedir, se, administración, que, el, en, co...","La administración concursal de Unipost, que ge...",La administración concursal de Unipost empezar...,La empresa se cerrará a finales del mes de abr...,2018-02-26
1,2,"[algo, superación, blogs, volver, últimos, tie...",La compañía española se mueve lenta pero consi...,Algo se mueve en Zardoya. Blogs de Bolságora,La compañía española se mueve lenta pero consi...,2018-01-29
2,3,"[le, tras, viral, primera, trasplante, que, un...","Jennifer Jones, de 40 años, es una mujer estad...",Virales: Respira por primera vez tras un trasp...,"Jennifer Jones, de 40 años, es una mujer estad...",2018-03-12
3,4,"[necesario, y, compras, los, en, la, rebaja, q...",¿Qué supone esta decisión para la economía de ...,Las claves de la rebaja en las compras del BCE...,La economía de la zona euro ha alcanzado el pu...,2017-10-27
4,5,"[presidente, como, blankfein, sus, schwartz, s...","Lloyd Bankfeild, consejero delegado de Goldman...",David Solomon se postula como el próximo presi...,"Lloyd Bankfeild, consejero delegado de Goldman...",2018-03-12


### 3.2. Summary statistics

First, we will need to filter out irrelevant words from the keywords list:

In [7]:
# Statistical summaries through dataset.describe() are not useful, since the only numeric variables are id and flag.
# Therefore, let's try to obtain some keyword frequencies as a first way to analyze the available information:

# Set stop words:
stopWords = list(stopwords.words('spanish'))

# List comprehension for stopword filtering:
trainKeywordList = []
testKeywordList = []

for instance in trainSet.keywords:
    trainKeywordList.append([x for x in instance.replace("[","").replace("]","").replace(",","").split(' ') if x not in stopWords])

# Append new column to the trainSet
#trainSet['KeywordList'] = pd.Series(trainKeywordList, index=trainSet.index)
    
for instance in testSet.keywords:
    testKeywordList.append([x for x in instance.replace("[","").replace("]","").replace(",","").split(' ') if x not in stopWords])
    
# Append new column to the testSet
#testSet['KeywordList'] = pd.Series(testKeywordList, index=testSet.index)

In [9]:
# We should also clean the titles and corpus of every article before analyzing them:
# 1. Word tokenization for title and corpus in train and test:
trainTitleToken = word_tokenize

'UniCredit duplica su beneficio en el primer trimestre, hasta los 907 millones de euros'

TODO:

- Veces que ha aparecido keyword anteriormente (total) + Veces por artículo. OJO temporalidad: hacer varias ventanas (1 dia, 3 días, 1 semana, 2 semanas, 1 mes, 3 meses, 6 meses, 1 año)

- Veces que ha aparecido keyword anteriormente en flag = 1 (total) + Veces por artículo. OJO temporalidad: hacer varias ventanas (1 dia, 3 días, 1 semana, 2 semanas, 1 mes, 3 meses, 6 meses, 1 año)

Resources:

- An introduction to text analysis in Python: https://www.digitalvidya.com/blog/an-introduction-to-text-analysis-in-python/
- Ultimate guide to deal with Text Data (using Python) – for Data Scientists & Engineers: https://www.analyticsvidhya.com/blog/2018/02/the-different-methods-deal-text-data-predictive-python/
- Online regex tester: https://regex101.com/